%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Internal Python packages
from datetime import datetime
import json
import os
import time
import typing
from typing import Tuple

# External Python packages
import boto3
import botocore
from botocore.exceptions import ClientError
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.externals import joblib
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

# Module files
import utils
from cosine_scheduler import CosineLRWithRestarts
from adamw import AdamW
from modules import Encoder, Decoder
from custom_types import DaRnnNet, TrainData, TrainConfig
from utils import numpy_to_tvar
from constants import device

In [ ]:
logger = utils.setup_log()
logger.info(f"Using computation device: {device}")

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, features, targets, window, time_start, time_end):
        self.window = window
        self.feats = len(features)
        
        df_model = df[features + targets].copy()
        
        # replace outliers with 0
        df_model[(np.abs(stats.zscore(df_model)) < 3).all(axis=1)] = 0
        df_model = df_model.fillna(0)
        index = df_model.loc[(df_model.index>time_start) & (df_model.index<time_end)].index
            
        df_x = df_model[features]
        df_y = df_model[targets]

        # Initialize scalers
        self.x_scaler = StandardScaler().fit(df_x.values)
        self.y_scaler = StandardScaler().fit(df_y.values)
        
        x_scaled = self.x_scaler.transform(df_x.values)
        x_scaled = pd.DataFrame(x_scaled, index=df_x.index)

        y_scaled = self.y_scaler.transform(df_y.values)
        y_scaled = pd.DataFrame(y_scaled, index=df_y.index)
        
        # filter df by start and end times
        train_X = x_scaled.loc[(x_scaled.index>time_start) & (x_scaled.index<time_end)].values
        train_y = y_scaled.loc[(y_scaled.index>time_start) & (y_scaled.index<time_end)].values
        
        self.data = TrainData(index, train_X, train_y)

    def __getitem__(self, index):
        x = self.data[index:index+self.window]
        # If there is a NaN in the sample, just return zeros
        if np.isnan(x).any():
            return torch.zeros(self.window, self.feats)
        else:
            return x

    def __len__(self):
        return self.data.feats.shape[0] - self.window
    
    
    def create_sample(self):
        idx = np.random.randint(self.data.feats.shape[0] - self.window)
        x = self.data.feats[idx:idx+self.window, :]
        y_history = self.data.targs[idx:idx+self.window-1, :]
        y_target = np.squeeze(self.data.targs[idx+self.window-1, :])
        return (x, y_history, y_target)
    
    
    def generate_batch(self, batch_size):
        """
        Generator function for creating random batches of training data.
        """

        # Infinite loop
        while True:
            # Allocate a new array for the batch of input signals
            x_batch = np.zeros((batch_size, self.window, self.data.feats.shape[1]))

            # Allocate a new array for the batch of output signals
            y_history_batch = np.zeros((batch_size, self.window - 1, self.data.targs.shape[1]))
            y_target_batch = np.zeros((batch_size, self.data.targs.shape[1]))

            # Fill the batch with random sequences of data
            for i in range(batch_size):

                # If the sequence contain any NaN, resample it
                contains_nan = True
                while contains_nan:
                    sample = self.create_sample()
                    x_batch[i, :, :] = sample[0]
                    y_history_batch[i, :, :] = sample[1]
                    y_target_batch[i, :] = sample[2]
                    contains_nan = np.isnan(sample[0]).any() or np.isnan(sample[1]).any() or np.isnan(sample[2]).any()

            yield (x_batch, y_history_batch, y_target_batch)
    

In [ ]:
def da_rnn(train_data: TrainData, encoder_hidden_size: int, decoder_hidden_size: int, T: int, batch_size: int):

    n_targs = train_data.data.targs.shape[1]
    train_cfg = TrainConfig(T, int(len(train_data) * 0.7), batch_size, nn.MSELoss())
    logger.info(f"Training size: {train_cfg.train_size:d}.")

    enc_kwargs = {"input_size": train_data.data.feats.shape[1], "hidden_size": encoder_hidden_size, "T": T}
    encoder = Encoder(**enc_kwargs).to(device)
    
    with open(os.path.join("data", "enc_kwargs.json"), "w") as fi:
        json.dump(enc_kwargs, fi, indent=4)

    dec_kwargs = {"encoder_hidden_size": encoder_hidden_size,
                  "decoder_hidden_size": decoder_hidden_size, "T": T, "out_feats": n_targs}
    decoder = Decoder(**dec_kwargs).to(device)
    
    with open(os.path.join("data", "dec_kwargs.json"), "w") as fi:
        json.dump(dec_kwargs, fi, indent=4)

    # Standard Adam
    #encoder_optimizer = optim.Adam(params=[p for p in encoder.parameters() if p.requires_grad], lr=learning_rate)
    #decoder_optimizer = optim.Adam(params=[p for p in decoder.parameters() if p.requires_grad], lr=learning_rate)
    
    # Adam with weight decay
    encoder_optimizer = AdamW(params=[p for p in encoder.parameters() if p.requires_grad], lr=1e-3, weight_decay=1e-4)
    decoder_optimizer = AdamW(params=[p for p in decoder.parameters() if p.requires_grad], lr=1e-3, weight_decay=1e-4)
    
    ## Only Cosine Annealing here
    epoch_size = train_cfg.train_size / batch_size
    encoder_lr_scheduler = CosineLRWithRestarts(encoder_optimizer, batch_size, epoch_size, restart_period=5, t_mult=1.2)
    decoder_lr_scheduler = CosineLRWithRestarts(decoder_optimizer, batch_size, epoch_size, restart_period=5, t_mult=1.2)
    
    #encoder_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(encoder_optimizer, patience=2)
    #decoder_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(decoder_optimizer, patience=2)
    
    da_rnn_net = DaRnnNet(encoder, decoder, encoder_optimizer, decoder_optimizer, encoder_lr_scheduler, decoder_lr_scheduler)

    return train_cfg, da_rnn_net


def train(net: DaRnnNet, train_data: TrainData, t_cfg: TrainConfig, n_epochs=10, save_plots=False):
    iter_per_epoch = int(np.ceil(t_cfg.train_size * 1. / t_cfg.batch_size))
    iter_losses = np.zeros(n_epochs * iter_per_epoch)
    epoch_losses = np.zeros((n_epochs, 2))
    logger.info(f"Iterations per epoch: {t_cfg.train_size * 1. / t_cfg.batch_size:3.3f} ~ {iter_per_epoch:d}.")
    
    # Create batch generator
    batch_gen = train_data.generate_batch(t_cfg.batch_size)
    
    n_iter = 0

    for e_i in range(n_epochs):
        
        # Loop through the data with steps of batch_size
        for t_i in range(0, t_cfg.train_size, t_cfg.batch_size):
            
            # Get the data for the set of batches
            feats, y_history, y_target = next(batch_gen)
            
            # Train one iteration and get the loss returned
            loss = train_iteration(net, t_cfg.loss_func, feats, y_history, y_target)

            # Save the loss
            iter_losses[e_i * iter_per_epoch + t_i // t_cfg.batch_size] = loss
            # if (j / t_cfg.batch_size) % 50 == 0:
            #    self.logger.info("Epoch %d, Batch %d: loss = %3.3f.", i, j / t_cfg.batch_size, loss)
            net.enc_lr.step()
            net.dec_lr.step()

            n_iter += 1


        y_test_pred = predict(net, train_data,
                                t_cfg.train_size, t_cfg.batch_size, t_cfg.T,
                                on_train=False)
            
        y_train_pred = predict(net, train_data,
                                t_cfg.train_size, t_cfg.batch_size, t_cfg.T,
                                on_train=True)
            
        val_loss = y_test_pred - train_data.data.targs[t_cfg.train_size:]
        epoch_losses[e_i, :] = [np.mean(iter_losses[range(e_i * iter_per_epoch, (e_i + 1) * iter_per_epoch)]),
                               np.mean(np.abs(val_loss))]
        logger.info(f"Epoch {(e_i+1):d}, train loss: {epoch_losses[e_i, 0]:3.6f}, val loss: {epoch_losses[e_i, 1]:3.6f}.")

        # Print som intermediate result every 5 epochs
        if (e_i + 1) % 5 == 0:
            
            plt.figure(figsize=(20,10))
            plt.plot(range(1, 1 + len(train_data.data.targs)), train_data.data.targs,
                     label="True")
            plt.plot(range(t_cfg.T, len(y_train_pred) + t_cfg.T), y_train_pred,
                     label='Predicted - Train')
            plt.plot(range(t_cfg.T + len(y_train_pred), len(train_data.data.targs) + 1), y_test_pred,
                     label='Predicted - Test')
            plt.legend(loc='upper left')
            utils.save_or_show_plot(f"pred_{e_i}.png", save_plots)

    return iter_losses, epoch_losses


def train_iteration(t_net: DaRnnNet, loss_func: typing.Callable, X, y_history, y_target):
    # Perform one training iteration for one batch
    
    # Zero the gradients
    t_net.enc_opt.zero_grad()
    t_net.dec_opt.zero_grad()

    # Forward through the encoder and decorder
    input_weighted, input_encoded = t_net.encoder(numpy_to_tvar(X))
    y_pred = t_net.decoder(input_encoded, numpy_to_tvar(y_history))

    # Calculate loss
    y_true = numpy_to_tvar(y_target)
    loss = loss_func(y_pred, y_true)
    
    # Take one step
    loss.backward()

    t_net.enc_opt.step()
    t_net.dec_opt.step()

    return loss.item()


def predict(t_net: DaRnnNet, t_dat: TrainData, train_size: int, batch_size: int, T: int, on_train=False):
    out_size = t_dat.data.targs.shape[1]
    if on_train:
        y_pred = np.zeros((train_size - T + 1, out_size))
    else:
        y_pred = np.zeros((t_dat.data.feats.shape[0] - train_size, out_size))

    # Loop through the y_pred vector in batches
    for y_i in range(0, len(y_pred), batch_size):
        # Create a slice in the y_pred tensor
        y_slc = slice(y_i, y_i + batch_size)
        # Get the corresponding incides for the slice
        batch_idx = range(len(y_pred))[y_slc]
        # Same as batch size?
        b_len = len(batch_idx)
        # X with size (batch size) x (time window) x (number of features)
        X = np.zeros((b_len, T, t_dat.data.feats.shape[1]))
        # y_history with size (batch size) x (time windows - 1) x (number of targets)
        y_history = np.zeros((b_len, T - 1, t_dat.data.targs.shape[1]))

        # Loop through each sample in the batch
        for b_i, b_idx in enumerate(batch_idx):
            # Get the indices for the data to get
            if on_train:
                idx_x = range(b_idx, b_idx + T )
                idx_yhist = range(b_idx, b_idx + T - 1)
            else:
                idx_x = range(b_idx + train_size - T, b_idx + train_size)
                idx_yhist = range(b_idx + train_size - T, b_idx + train_size - 1)

            X[b_i, :, :] = t_dat.data.feats[idx_x, :]
            y_history[b_i, :] = t_dat.data.targs[idx_yhist]

        y_history = numpy_to_tvar(y_history)
        _, input_encoded = t_net.encoder(numpy_to_tvar(X))
        y_pred[y_slc] = t_net.decoder(input_encoded, y_history).cpu().data.numpy()

    return y_pred


def predict_df(df, features, target_cols, t_net: DaRnnNet, batch_size: int, T: int, date_start, date_end):
    
    #date_start = df.index[0]
    #date_end = df.index[-1]
    t_dat = MyDataset(df, features, target_cols, T, date_start, date_end)
    y_scaler = t_dat.y_scaler
    
    out_size = t_dat.data.targs.shape[1]
    y_pred = np.zeros((t_dat.data.index.shape[0], out_size))
    
    # Loop through the y_pred vector in batches
    for y_i in range(T, len(y_pred), batch_size):
        
        # Create a slice in the y_pred tensor
        y_slc = slice(y_i, y_i + batch_size)
        
        # Get the corresponding incides for the slice
        batch_idx = range(len(y_pred))[y_slc]
        
        # Same as batch size?
        b_len = len(batch_idx)

        # X with size (batch size) x (time window) x (number of features)
        X = np.zeros((b_len, T, t_dat.data.feats.shape[1]))
        
        # y_history with size (batch size) x (time windows - 1) x (number of targets)
        y_history = np.zeros((b_len, T - 1, t_dat.data.targs.shape[1]))

        # Loop through each sample in the batch
        for b_i, b_idx in enumerate(batch_idx):
            
            # Get the indices for the data to get
            idx_x = range(b_idx - T, b_idx)
            idx_yhist = range(b_idx - T, b_idx - 1)

            X[b_i, :, :] = t_dat.data.feats[idx_x, :]
            y_history[b_i, :] = t_dat.data.targs[idx_yhist]

        y_history = numpy_to_tvar(y_history)
        _, input_encoded = t_net.encoder(numpy_to_tvar(X))
        y_pred[y_slc] = t_net.decoder(input_encoded, y_history).cpu().data.numpy()
        
    new_cols = [col+'_pred' for col in target_cols]
    y_pred_inv = y_scaler.inverse_transform(y_pred)
    df_y_pred = pd.DataFrame(data=y_pred_inv, index=t_dat.data.index, columns=new_cols)
    
    df = pd.merge(df, df_y_pred, left_index=True, right_index=True, how='left')

    return df


In [ ]:
save_plots = True
debug = False

target_cols = [
    'col1'
    ]
features = [
    'col2'
]

train_start = datetime(2013, 1, 1)
train_end = datetime(2014, 12, 31)
test_start = datetime(2016, 1, 1)
test_end = datetime(2016, 6, 13)

data_train = MyDataset(df, features, target_cols, 12, train_start, train_end)
x_scaler_train = data_train.x_scaler
y_scaler_train = data_train.y_scaler
data_test = MyDataset(df, features, target_cols, 12, test_start, test_end)
x_scaler_test = data_test.x_scaler
y_scaler_test = data_test.y_scaler

In [ ]:
from modules import Encoder, Decoder

da_rnn_kwargs = {"batch_size": 128, "T": 12, "encoder_hidden_size": 128, "decoder_hidden_size": 128}
config, model = da_rnn(data_train, **da_rnn_kwargs)
iter_loss, epoch_loss = train(model, data_train, config, n_epochs=30, save_plots=save_plots)

plt.figure(figsize=(12, 6))
plt.semilogy(range(len(iter_loss)), iter_loss)
utils.save_or_show_plot("iter_loss.png", save_plots)

plt.figure(figsize=(12, 6))
plt.semilogy(range(epoch_loss.shape[0]), epoch_loss[:, 0], label='Train loss')
plt.semilogy(range(epoch_loss.shape[0]), epoch_loss[:, 1], label='Test loss')
plt.legend(loc='upper right')
utils.save_or_show_plot("epoch_loss.png", save_plots)

final_y_pred_train = predict(model, data_train, config.train_size, config.batch_size, config.T, on_train=True)
final_y_pred_test = predict(model, data_train, config.train_size, config.batch_size, config.T, on_train=False)

plt.figure(figsize=(12, 6))
plt.plot(range(1, 1 + len(data_train.data.targs)), data_train.data.targs, label="True")
plt.plot(range(config.T, len(final_y_pred_train) + config.T), final_y_pred_train, label='Predicted - Train')
plt.plot(range(config.T + len(final_y_pred_train), len(data_train.data.targs) + 1), final_y_pred_test, label='Predicted - Test')
plt.legend(loc='upper left')

In [ ]:
import bokeh.plotting
from bokeh.models import LinearAxis, Range1d
from bokeh.plotting import output_notebook; output_notebook(notebook_type='jupyter') 

# Calculate maximal error in train data
df_train_preds = predict_df(df, features, target_cols, model, config.batch_size, config.T, train_start, train_end)
df_train_preds = df_train_preds[train_start:train_end].dropna()

max_errors = {}
for target_col in target_cols: 
    df_train_preds[target_col+'_error'] = df_train_preds[target_col] - df_train_preds[target_col+'_pred']
    max_error = np.percentile(np.abs(df_train_preds[target_col+'_error']), 99.5)
    max_errors[target_col] = max_error

# Make prediction for a new time span
date_start = datetime(2014, 1, 1)
date_end = datetime(2016, 6, 13)
df_test = predict_df(df, features, target_cols, model, config.batch_size, config.T, date_start, date_end)
df_test = df_test[date_start:date_end]

for target_col in target_cols: 
    df_test[target_col+'_error'] = df_test[target_col] - df_test[target_col+'_pred']
    df_test[target_col+'_anomaly'] = 0
    df_test.loc[np.abs(df_test[target_col+'_error']) >= max_errors[target_col], target_col+'_anomaly'] = 1
    df_test[target_col+'_cum_anomaly'] = df_test[target_col+'_anomaly'].cumsum()



p = bokeh.plotting.figure(title='test', x_axis_label='x', y_axis_label='y', plot_width=1400, plot_height=1000, y_range=(0, 70), x_axis_type="datetime")
p.extra_y_ranges = {'foo': Range1d(start=0, end=1000)}
p.add_layout(LinearAxis(y_range_name="foo"), 'right')
for target_col in target_cols:
    p.line(df_test.index, df_test[target_col], legend="True", line_width=1, line_color='blue')
    p.line(df_test.index, df_test[target_col+'_pred'], legend="Predicted", line_width=1, line_color='red')
    p.line(df_test.index, df_test[target_col+'_cum_anomaly'], legend="Cum. Anomaly", line_width=1, line_color='green', y_range_name='foo')

bokeh.plotting.show(p)